In [1]:
from matplotlib import pyplot
import tensorflow as tf
import tensorflow.keras as keras
from tensorflow.keras.datasets import cifar10


(X_train, y_train), (X_test, y_test) = cifar10.load_data()
y_train = tf.keras.utils.to_categorical(y_train)
y_test = tf.keras.utils.to_categorical(y_test)

In [2]:
print(X_train.shape)
print(y_train.shape)

print(X_test.shape)
print(y_test.shape)

print(y_test[0])

(50000, 32, 32, 3)
(50000, 10)
(10000, 32, 32, 3)
(10000, 10)
[0. 0. 0. 1. 0. 0. 0. 0. 0. 0.]


In [3]:
def padding_layer_iyswim(inputs, shape, name=None):
    h_start = shape[0]
    w_start = shape[1]
    output_short = shape[2]
    input_shape = tf.shape(inputs)
    input_short = tf.reduce_min(input_shape[1:3])
    input_long = tf.reduce_max(input_shape[1:3])
    output_long = tf.to_int32(tf.ceil(
        1. * tf.to_float(output_short) * tf.to_float(input_long) / tf.to_float(input_short)))
    output_height = tf.to_int32(input_shape[1] >= input_shape[2]) * output_long +\
        tf.to_int32(input_shape[1] < input_shape[2]) * output_short
    output_width = tf.to_int32(input_shape[1] >= input_shape[2]) * output_short +\
        tf.to_int32(input_shape[1] < input_shape[2]) * output_long
    return tf.pad(inputs, tf.to_int32(tf.stack([[0, 0], [h_start, output_height - h_start - input_shape[1]], [w_start, output_width - w_start - input_shape[2]], [0, 0]])), name=name)

In [4]:
from PIL import Image
from os import listdir
from os.path import isfile, join

mypath = "./adversarial_poisons/img/data"
onlyfiles = [f for f in listdir(mypath) if isfile(join(mypath, f))]

onlyfiles = sorted(onlyfiles, key = lambda x: int(x[:-4]))

In [5]:
print(len(onlyfiles))

50000


In [6]:
model = tf.keras.applications.vgg16.VGG16(weights=None,
    input_shape=(32,32,3), pooling=None, classes=10,
    classifier_activation='softmax'
)

#sgd = optimizers.SGD(lr=learning_rate, decay=lr_decay, momentum=0.9, nesterov=True)
#        model.compile(loss='categorical_crossentropy', optimizer=sgd,metrics=['accuracy'])

model.compile(optimizer='SGD', \
              loss='categorical_crossentropy',\
              metrics=["acc"])
model.summary()

2021-11-21 00:29:24.819235: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2021-11-21 00:29:25.480370: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1510] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 9659 MB memory:  -> device: 0, name: GeForce RTX 2080 Ti, pci bus id: 0000:61:00.0, compute capability: 7.5


Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 32, 32, 3)]       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 32, 32, 64)        1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 32, 32, 64)        36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 16, 16, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 16, 16, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 16, 16, 128)       147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 8, 8, 128)         0     

In [7]:
from keras.callbacks import EarlyStopping, ModelCheckpoint

CHECKPOINT_PATH = 'best.hdf5'

es = EarlyStopping(monitor = 'val_loss', mode = 'min', verbose = 1, patience = 5)
cp = ModelCheckpoint(CHECKPOINT_PATH, monitor = 'val_loss', mode = 'min', verbose = 1, save_best_only = True)

history = model.fit(X_train, y_train, epochs = 1000, validation_split = 0.1, callbacks=[es,cp])
model.save('saved_model/vgg16_32_32_ep100_val10per')

2021-11-21 00:29:26.709239: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


Epoch 1/1000


2021-11-21 00:29:27.900794: I tensorflow/stream_executor/cuda/cuda_dnn.cc:369] Loaded cuDNN version 8201
2021-11-21 00:29:28.520835: I tensorflow/core/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2021-11-21 00:29:28.521618: I tensorflow/core/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2021-11-21 00:29:28.521633: W tensorflow/stream_executor/gpu/asm_compiler.cc:77] Couldn't get ptxas version string: Internal: Couldn't invoke ptxas --version
2021-11-21 00:29:28.523651: I tensorflow/core/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2021-11-21 00:29:28.523726: W tensorflow/stream_executor/gpu/redzone_allocator.cc:314] Internal: Failed to launch ptxas
Relying on driver to perform ptx compilation. 
Modify $PATH to customize ptxas location.
This message will be only logged once.


1407/1407 [==============================] - 34s 21ms/step - loss: 1.9362 - acc: 0.2875 - val_loss: 1.6995 - val_acc: 0.3558

Epoch 00001: val_loss improved from inf to 1.69952, saving model to best.hdf5
Epoch 2/1000
1407/1407 [==============================] - 28s 20ms/step - loss: 1.4832 - acc: 0.4607 - val_loss: 1.3761 - val_acc: 0.4918

Epoch 00002: val_loss improved from 1.69952 to 1.37608, saving model to best.hdf5
Epoch 3/1000
1407/1407 [==============================] - 28s 20ms/step - loss: 1.2166 - acc: 0.5645 - val_loss: 1.7145 - val_acc: 0.3636

Epoch 00003: val_loss did not improve from 1.37608
Epoch 4/1000
1407/1407 [==============================] - 28s 20ms/step - loss: 1.0126 - acc: 0.6409 - val_loss: 1.5042 - val_acc: 0.4460

Epoch 00004: val_loss did not improve from 1.37608
Epoch 5/1000
1407/1407 [==============================] - 28s 20ms/step - loss: 0.8525 - acc: 0.7033 - val_loss: 0.9680 - val_acc: 0.6636

Epoch 00005: val_loss improved from 1.37608 to 0.96803, 

2021-11-21 00:36:05.003561: W tensorflow/python/util/util.cc:348] Sets are not currently considered sequences, but this may change in the future, so consider avoiding using them.


INFO:tensorflow:Assets written to: saved_model/vgg16_32_32_ep100_val10per/assets
